In [8]:
# Kết nối Google drive để lưu tài nguyên
from google.colab import drive
drive.mount('/content/drive')

# Chuyển đến thư mục
%cd /content/drive/My Drive/Individual AI Project/crypto

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Individual AI Project/crypto


In [9]:
import yfinance as yf

from datetime import datetime
import time

import pandas as pd
import numpy as np

In [10]:
class YahooDailyReader():
    def __init__(self, symbol=None, start=None, end=None):
        """
        Khởi tạo YahooDailyReader với mã chứng khoán và khoảng thời gian.

        Tham số:
        symbol (str): Mã cổ phiếu, ví dụ: 'AAPL' cho Apple.
        start (datetime): Ngày bắt đầu để lấy dữ liệu lịch sử.
        end (datetime): Ngày kết thúc để lấy dữ liệu lịch sử.
        """

        self.symbol = symbol
        self.start = start
        self.end = end

        # Chuyển đổi ngày bắt đầu và ngày kết thúc sang định dạng Unix timestamp.
        unix_start = int(time.mktime(self.start.timetuple()))
        day_end = self.end.replace(hour=23, minute=59, second=59)
        unix_end = int(time.mktime(day_end.timetuple()))

    def read(self):
        """
        Lấy và phân tích dữ liệu lịch sử từ Yahoo Finance.

        Trả về:
        pandas.DataFrame: Một DataFrame chứa dữ liệu giá lịch sử.
        """

        # Lấy dữ liệu từ Yahoo Finance.
        stock_data = yf.download(self.symbol, start=self.start, end=self.end, interval='1d')

        # Chuyển đổi dữ liệu JSON thành một pandas DataFrame.
        df = pd.DataFrame(stock_data).reset_index()

         # Thêm mã cổ phiếu như một cột trong DataFrame.
        df.insert(1, 'symbolid', self.symbol)

        # Chuyển đổi Unix timestamps thành date objects.
        df['Date'] = pd.to_datetime(df['Date'], unit='s').dt.date

        # Loại bỏ các hàng mà giá 'Close' là NaN.
        df = df.dropna(subset=['Close'])

        # Đổi tên các cột.
        colnames = ['Date', 'symbolid', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']
        df.columns = colnames

        # Đặt cột 'Date' làm index.
        df = df.set_index('Date')

        return df

In [11]:
symbol_list=['BTC-USD', 'ETH-USD', 'ADA-USD']

In [12]:
start = datetime(1990, 1, 1)
end = datetime(2024, 6, 20)
df_list=[]
for symbol in symbol_list:
    reader = YahooDailyReader(symbol, start, end)
    dfs = reader.read()
    dfs.to_csv(f'csvdata/{symbol}.csv')
    df_list.append(dfs)

df = pd.concat(df_list)
df.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,symbolid,Open,High,Low,Close,AdjClose,Volume
Date,,,,,,,
2014-09-17,BTC-USD,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,BTC-USD,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,BTC-USD,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,BTC-USD,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,BTC-USD,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [13]:
df.to_csv('csvdata/all_data.csv')